In [ ]:
!git clone https://github.com/aashishkhimasia/google-research

In [ ]:
%pip install -r /content/google-research/requirements.txt

In [ ]:
import tensorflow as tf

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
print('Running on TPU ', tpu.cluster_spec().as_dict())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/google-research

In [ ]:
!tar -xzf "gs://dissertation-adk/pretrained_emb_voc/vision_vatt_misc_data.tgz" -C "/content/google-research/vatt/misc/"     #[run this cell to extract tar.gz files]

In [ ]:
# get the file in, see if that helps with the howto100m

In [ ]:
!tar -xvzf "gs://dissertation-adk/pretrained_emb_voc/vision_vatt_misc_data.tgz" --force-local


Main.py from vatt (with edits from LG)

In [ ]:
import pprint

# from absl import app
# from absl import flags
from absl import logging

from vatt.configs import factory as config_factory
from vatt.experiments import finetune
from vatt.experiments import pretrain

# flags.DEFINE_string('task', 'PRETRAIN', 'PRETRAIN or FINETUNE.')
# flags.DEFINE_string('mode', 'train', 'train or eval.')
# flags.DEFINE_string('model_dir', None, 'Default path for the experiment.')
# flags.DEFINE_string('model_arch', 'Tx_FAC', 'Arch of the model.')
# flags.DEFINE_string('override_checkpoint', None,
#                     ('Path to a checkpoint for initialization. '
#                      'If this is passed, the model is initialized from this '
#                      'checkpoint, even if there is a valid latest checkpoint '
#                      'inside the model_dir.'))
# flags.DEFINE_string('config_file', None,
#                     ('Path to a YAML config file containing the dictionary of '
#                      'parameters to override the original params defined '
#                      'under configs/'))
# flags.DEFINE_string('params_override', None,
#                     'A safe_dumped str of a dictionary of parameters')
# flags.DEFINE_string('strategy_type', 'tpu', 'Type of the distribution strategy')
# flags.DEFINE_string('tpu', None, 'Address of the TPU device')


# FLAGS = flags.FLAGS


def get_params():
  """Constructs the configuration of the experiment."""

  task = "FINETUNE"  # FLAGS.task
  model_arch = "Tx_FAC"  # FLAGS.model_arch
  params = config_factory.build_experiment_configs(
      task=task,
      model_arch=model_arch,
      )

  # if FLAGS.config_file:
  #   params.override_from_file(FLAGS.config_file)

 # if FLAGS.params_override:
 #    params.override_from_str(FLAGS.params_override)

  params.override({
      'mode': "eval",  # FLAGS.mode,
      'model_dir': None, # FLAGS.model_dir,
      'checkpoint_path': 'gs://dissertation-adk/checkpoint', #None, # FLAGS.override_checkpoint,
      'strategy_config': {'tpu': None, # FLAGS.tpu,
                          'distribution_strategy': "tpu" # FLAGS.strategy_type
                          },
  })

  return params



def main():
  # del argv  # Unused.

  params = get_params()
  logging.info('Model Parameters: %s', pprint.pformat(params.as_dict()))

  print(str(params.model_dir))
  params.eval.input.name = "toy_ds"
  if params.task.lower() == 'pretrain':
    executor = pretrain.get_executor(params=params)

  elif params.task.lower() == 'finetune':
    executor = finetune.get_executor(params=params)

  else:
    raise ValueError('Task not found: %s.' % params.task)

  return executor.run(mode=params.mode)


if __name__ == '__main__':
  # app.run(main)
  main()

In [ ]:
%ls